In [1]:
import torch
from torchvision import transforms,datasets  #transform针对图像处理
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim


In [2]:
#1 prepare dataset
batch_size = 64
transform = transforms.Compose([
 transforms.ToTensor(),
 transforms.Normalize((0.1307, ), (0.3081, )) #把图像转化成张量，并且将其标准化使得成为一个N(0,1)的张量
])
train_dataset = datasets.MNIST(root='../dataset/mnist/',
train=True,
download=True,
transform=transform)

train_loader = DataLoader(train_dataset,
shuffle=True,
batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist/',
train=False,
download=True,
transform=transform)

test_loader = DataLoader(test_dataset,
shuffle=False,
batch_size=batch_size)

#至此，完成了图像载入，并且把图像转化成为适合NN训练的N（0，1）的张量

In [3]:
#02 design the model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
    
model=Net()

In [4]:
#03 construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [5]:
#04 training cycle
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
# forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss=0.0

In [6]:
#05  test
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%' % (100 * correct /total)
          


SyntaxError: unexpected EOF while parsing (3449140717.py, line 13)

In [ ]:
train(1)
test()